# Parse / Extract consistent needed data

In [36]:
import pandas as pd
from pathlib import Path

## Parse Kim Excel

In [ ]:
# Define paths
DATA_PATH = Path("..") / "data"
INPUT_PATH = DATA_PATH / "raw" / "2023_Kim.xlsx"
OUTPUT_PATH = DATA_PATH / "interim" / "toxprot_2017_old.csv"

# Define constants
SHEET_NAME = "ToxProt11.2017"
COLUMNS_TO_EXTRACT = [
    "Entry",
    "Organism",
    "Protein families",
    "Length (aa)",
    "Fragments",
    "Toxic dose",
    "PTM",
]

# Read specific columns from the Excel sheet
df = pd.read_excel(INPUT_PATH, sheet_name=SHEET_NAME, usecols=COLUMNS_TO_EXTRACT)

# Rename 'Length (aa)' to 'Length' and 'Fragments' to 'Fragment'
df = df.rename(columns={"Length (aa)": "Length", "Fragments": "Fragment"})

# Ensure output directory exists and save as CSV
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUTPUT_PATH, index=False)

# Display information about the processed data
print(f"Data extracted and saved to {OUTPUT_PATH}")
print(f"Shape of the extracted data: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("First few rows:")
display(df.head())


## Parse SwissProt 2017-11 (extracted from [FTP](https://ftp.uniprot.org/pub/databases/uniprot/previous_major_releases/release-2017_11/))

In [38]:
def update_protfams(df):
    # Split on common delimiters and take first part
    df["Protein families"] = df["Protein families"].str.split(r"[.,;]").str[0]
    print(
        f"Unique protein families after splitting: {df['Protein families'].nunique()}"
    )

    # Map of family name corrections
    family_corrections = {
        "I1 superfamily": "Conotoxin I1 superfamily",
        "O1 superfamily": "Conotoxin O1 superfamily",
        "O2 superfamily": "Conotoxin O2 superfamily",
        "E superfamily": "Conotoxin E superfamily",
        "F superfamily": "Conotoxin F superfamily",
        "Conotoxin M family": "Conotoxin M superfamily",
        "Conotoxin B2 family": "Conotoxin B2 superfamily",
        "Conotoxin O1 family": "Conotoxin O1 superfamily",
        "Conotoxin O2 family": "Conotoxin O2 superfamily",
    }

    # Apply all corrections at once
    df["Protein families"] = df["Protein families"].replace(family_corrections)
    print(
        f"Unique protein families after processing: {df['Protein families'].nunique()}"
    )

    return df

In [ ]:
# Define paths for the TSV file
TSV_INPUT_PATH = DATA_PATH / "interim" / "toxprot_2017.tsv"
CAV_OUTPUT_PATH = DATA_PATH / "interim" / "toxprot_2017.csv"

# Define columns to extract with the correct column name for PTM
COLUMNS_TO_EXTRACT = [
    "Entry",
    "Organism",
    "Organism (ID)",
    "Protein families",
    "Length",
    "Fragment",
    "Toxic dose",
    "Post-translational modification",  # Correct column name in the TSV file
]

# Read the TSV file
df_tsv = pd.read_csv(TSV_INPUT_PATH, sep="\t", usecols=COLUMNS_TO_EXTRACT)

# Rename 'Post-translational modification' to 'PTM' for consistency
df_tsv = df_tsv.rename(columns={"Post-translational modification": "PTM"})

# update protein families
df_tsv = update_protfams(df_tsv)

# Save as CSV
df_tsv.to_csv(CAV_OUTPUT_PATH, index=False)

# Display information about the processed data
print(f"TSV data extracted and saved to {CAV_OUTPUT_PATH}")
print(f"Shape of the extracted TSV data: {df_tsv.shape}")
print(f"Columns: {df_tsv.columns.tolist()}")
print("First few rows:")
display(df_tsv.head())

## Parse SwissProt 2025-03

In [ ]:
# Define paths for the TSV file
TSV_INPUT_PATH = DATA_PATH / "raw" / "202503_ToxProt.tsv"
CSV_OUTPUT_PATH = DATA_PATH / "interim" / "toxprot_2025.csv"

# Define columns to extract with the correct column name for PTM
COLUMNS_TO_EXTRACT = [
    "Entry",
    "Organism",
    "Organism (ID)",
    "Protein families",
    "Length",
    "Fragment",
    "Toxic dose",
    "Post-translational modification",  # Correct column name in the TSV file
]

# Read the TSV file
df_tsv = pd.read_csv(TSV_INPUT_PATH, sep="\t", usecols=COLUMNS_TO_EXTRACT)

# Rename 'Post-translational modification' to 'PTM' for consistency
df_tsv = df_tsv.rename(columns={"Post-translational modification": "PTM"})

# update protein families
df_tsv = update_protfams(df_tsv)

# Save as CSV
df_tsv.to_csv(CSV_OUTPUT_PATH, index=False)

# Display information about the processed data
print(f"TSV data extracted and saved to {CSV_OUTPUT_PATH}")
print(f"Shape of the extracted TSV data: {df_tsv.shape}")
print(f"Columns: {df_tsv.columns.tolist()}")
print("First few rows:")
display(df_tsv.head())


# Get taxonomy info
Process taxonomic information from a CSV file using taxopy.

In [28]:
import pandas as pd
import taxopy
from pathlib import Path


def setup_db_paths():
    """Setup and return the database paths."""
    home_dir = Path.home() / ".cache"
    db_dir = home_dir / "taxopy_db"
    db_dir.mkdir(parents=True, exist_ok=True)
    nodes_file = db_dir / "nodes.dmp"
    names_file = db_dir / "names.dmp"
    merged_file = db_dir / "merged.dmp"

    return db_dir, nodes_file, names_file, merged_file


def initialize_taxdb():
    """Initialize and return the taxonomy database."""
    # Get the database paths
    db_dir, nodes_file, names_file, merged_file = setup_db_paths()

    if nodes_file.exists() and names_file.exists():
        print(f"Loading existing taxopy database from {db_dir}")
        taxdb = taxopy.TaxDb(
            nodes_dmp=str(nodes_file),
            names_dmp=str(names_file),
            merged_dmp=str(merged_file),
        )
    else:
        print(f"Downloading taxopy database to {db_dir}")
        taxdb = taxopy.TaxDb(taxdb_dir=str(db_dir), keep_files=True)

    return taxdb


def get_taxonomy_info(taxon_id, taxdb):
    """Get order, family, genus info for a taxon ID."""
    # Get the Taxon object
    taxon = taxopy.Taxon(taxon_id, taxdb)

    # Get the rank information
    ranks = taxon.rank_name_dictionary

    return {
        "taxon_name": taxon.name,
        "order": ranks.get("order", ""),
        "family": ranks.get("family", ""),
        "genus": ranks.get("genus", ""),
    }  # superkingdom, kingdom, phylum, class, order, family, genus, species


def build_taxonomy_cache(df, taxdb):
    """Build a cache of taxonomy information for all unique organism IDs."""
    taxonomy_cache = {}
    for taxon_id in df["Organism (ID)"].unique():
        if pd.notna(taxon_id):
            taxonomy_cache[taxon_id] = get_taxonomy_info(taxon_id, taxdb)

    return taxonomy_cache


def add_taxonomy_columns(df, taxonomy_cache):
    """Add taxonomy columns to the dataframe."""

    # Create a mapping function that extracts all taxonomy info at once
    def get_taxonomy_info(taxon_id):
        cache_entry = taxonomy_cache.get(taxon_id, {})
        return pd.Series(
            {
                "Scientific_Name": cache_entry.get("taxon_name", ""),
                "Order": cache_entry.get("order", ""),
                "Family": cache_entry.get("family", ""),
                "Genus": cache_entry.get("genus", ""),
            }
        )

    # Apply the mapping function once to get all columns
    taxonomy_df = df["Organism (ID)"].apply(get_taxonomy_info)

    # Concatenate the new columns with the original dataframe
    return pd.concat([df, taxonomy_df], axis=1)


def process_dataframe(input_path, output_path, taxdb):
    """Process the dataframe: load, add taxonomy, remove Organism column, save."""
    # Load the dataframe
    print(f"Loading data from {input_path}")
    df = pd.read_csv(input_path)

    # Build the taxonomy cache
    print("Building taxonomy cache...")
    taxonomy_cache = build_taxonomy_cache(df, taxdb)

    # Add taxonomy columns
    print("Adding taxonomy columns...")
    df = add_taxonomy_columns(df, taxonomy_cache)

    # Remove the Organism column if it exists
    if "Organism" in df.columns:
        print("Removing 'Organism' column...")
        df = df.drop(columns=["Organism"])

    # Save the updated dataframe
    print(f"Saving processed data to {output_path}")
    df.to_csv(output_path, index=False)
    print(f"Processing complete. Data saved to {output_path}")

In [ ]:
print("Initializing taxonomy database...")
taxdb = initialize_taxdb()

# Process ToxProt 2017-11
input_path = "../data/interim/toxprot_2017.csv"
output_path = "../data/processed/toxprot_2017.csv"
process_dataframe(input_path, output_path, taxdb)

# Process ToxProt 2025-03
input_path = "../data/interim/toxprot_2025.csv"
output_path = "../data/processed/toxprot_2025.csv"
process_dataframe(input_path, output_path, taxdb)

# Differentiate between marine and terrestrial organism

In [ ]:
import json

# Load the marine/terrestrial mapping
mapping_path = "../data/raw/marine_terrestrial.json"

# Load the marine/terrestrial mapping
with open(mapping_path, "r") as f:
    habitat_mapping = json.load(f)


# Function to determine habitat based on order and genus
def determine_habitat(row):
    order = row["Order"]
    genus = row.get("Genus", "")  # Get genus if available, otherwise empty string

    # Check if order is in clear_orders
    if order in habitat_mapping["clear_orders"]["terrestrial"]:
        return "terrestrial"
    elif order in habitat_mapping["clear_orders"]["marine"]:
        return "marine"

    # Check if order is in ambiguous_orders
    if order in habitat_mapping["ambiguous_orders"]:
        # Check if genus is in the terrestrial list for this order
        if genus in habitat_mapping["ambiguous_orders"][order].get("terrestrial", {}):
            return "terrestrial"
        # Check if genus is in the marine list for this order
        elif genus in habitat_mapping["ambiguous_orders"][order].get("marine", {}):
            return "marine"

    # If we can't determine, return 'unknown'
    return "unknown"


# Process 2017 dataset
csv_path_2017 = "../data/processed/toxprot_2017.csv"
df_2017 = pd.read_csv(csv_path_2017)
df_2017["Habitat"] = df_2017.apply(determine_habitat, axis=1)
print("ToxProt 2017 habitat distribution:")
print(df_2017["Habitat"].value_counts())
df_2017.to_csv(csv_path_2017, index=False)
print(f"Updated {csv_path_2017} with habitat information")

# Process 2025 dataset
csv_path_2025 = "../data/processed/toxprot_2025.csv"
df_2025 = pd.read_csv(csv_path_2025)
df_2025["Habitat"] = df_2025.apply(determine_habitat, axis=1)
print("\nToxProt 2025 habitat distribution:")
print(df_2025["Habitat"].value_counts())
df_2025.to_csv(csv_path_2025, index=False)
print(f"Updated {csv_path_2025} with habitat information")
